Eye Blink Based Secured Door Lock

In [ ]:
from cvzone import putTextRect, cornerRect
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot
from cvzone.SerialModule import SerialObject
from cvzone.FPS import FPS
import cv2
import time 

# Initialize the webcam
# '2' indicates the third camera connected to the computer, '0' would usually refer to the built-in webcam
cap = cv2.VideoCapture(0)

arduino = SerialObject("COM7", digits=1) 
fpsReader = FPS(avgCount=30)

# Initialize FaceMeshDetector object
# staticMode: If True, the detection happens only once, else every frame
# maxFaces: Maximum number of faces to detect
# minDetectionCon: Minimum detection confidence threshold
# minTrackCon: Minimum tracking confidence threshold


detector = FaceMeshDetector(staticMode=False, maxFaces=1, minDetectionCon=0.5, minTrackCon=0.5)
plotY = LivePlot(640, 360, [20, 50])
#idList = [22, 23, 24, 26, 118, 157, 158, 159, 160, 161, 130, 243]
idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList=[] 
blinkCounter = 0
flag = 0
color = (0, 0, 255)

# Timer variables
timer_started = False
start_time = 0
flagTimer= True 

# Start the loop to continually get frames from the webcam
while True:
    # Read the current frame from the webcam
    # success: Boolean, whether the frame was successfully grabbed
    # img: The current frame
    success, img = cap.read()
    H,W,_ = img.shape

    #roix1, roiy1, roix2, roiy2 = int(W*0.35), int(H*0.01), int(W-0.35*W), int(H-0.5*H)
    roix1, roiy1, roix2, roiy2 = int(W*0.35), int(H*0.05), int(W-0.35*W), int(H-0.4*H)
    imgroi = img[roiy1:roiy2, roix1:roix2]
    cornerRect(img, [roix1, roiy1, roix2-roix1, roiy2-roiy1],rt=2,colorR=(0,0,0),colorC=(255,0,0))
    #cv2.imshow("Check", img[roiy1:roiy2+5,roix1-5:roix2+5])

    # Find face mesh in the image
    # img: Updated image with the face mesh if draw=True
    # faces: Detected face information
    imgroi, faces = detector.findFaceMesh(imgroi, draw=False)
    

    # Check if any faces are detected
    if faces:
            face = faces[0]

            for id in idList: 
                  cv2.circle(img, (face[id][0]+roix1, face[id][1]+roiy1), 2, color, cv2.FILLED)
            # Get specific points for the eye
            # leftEyeUpPoint: Point above the left eye
            # leftEyeDownPoint: Point below the left eye
            leftEyeUpPoint = face[159]
            leftEyeDownPoint = face[23] #23

            leftEyeLeftPoint = face[130]
            leftEyeRightPoint = face[243]

            # Assume P1, P2, P3, P4 are defined as (x, y) tuples
            points = [leftEyeUpPoint, leftEyeDownPoint, leftEyeLeftPoint, leftEyeRightPoint]
            adjusted_points = []

            for point in points:
                x, y = point
                x_adjusted = x + roix1
                y_adjusted = y + roiy1
                adjusted_points.append((x_adjusted, y_adjusted))

            leftEyeUpPoint, leftEyeDownPoint, leftEyeLeftPoint, leftEyeRightPoint = adjusted_points

            # cv2.line(img, leftEyeDownPoint,leftEyeUpPoint, (255,0,0),2)
            # cv2.line(img, leftEyeLeftPoint,leftEyeRightPoint, (0,255,0),2)

            #print(face)

            # Calculate the vertical distance between the eye points
            # leftEyeVerticalDistance: Distance between points above and below the left eye
            # info: Additional information (like coordinates)
            leftEyeVerticalDistance, info = detector.findDistance(leftEyeUpPoint, leftEyeDownPoint)
            leftEyeHorizontalDistance, info = detector.findDistance(leftEyeLeftPoint, leftEyeRightPoint)
            ratio = (leftEyeVerticalDistance/leftEyeHorizontalDistance)*100

            ratioList.append(ratio)
            if len(ratioList)>5: 
                 ratioList.pop(0)

            ratioAvg = sum(ratioList)/len(ratioList)

            if ratioAvg < 30 and flag == 0:  #33
                 imgPlot = plotY.update(ratioAvg, color= (0,255,0))
                 blinkCounter = blinkCounter+1
                 flag=1
                 color = (0,210,0)   

            if not timer_started and blinkCounter:
                timer_started = True
                start_time = time.time() 

            if flag!=0:
                 flag+=1
                 if flag>10:
                      flag=0
                      color = (0,0,255)

            # Check timer
            if flagTimer and timer_started and (time.time() - start_time > 5):
                flagTimer = False

                if blinkCounter == 3:

                    arduino.sendData([1])
                    print("Success")
    
                else:
                    arduino.sendData([2])
                    print("Failure")
                    
                   
                


            putTextRect(img, f'Blink Count: {blinkCounter}',(430, 65), scale=1.5, 
                             colorR= color, colorB = (255,255,0), offset=8, thickness=2)


                 
            # Print the vertical distance for debugging or information
            #print(ratio)
            #putTextRect(img, f'Distance: {int(ratio)}cm', (face[10][0] - 100, face[10][1] - 50), scale=2)
            imgPlot = plotY.update(ratioAvg, color)
            cv2.imshow("Blink Plot", imgPlot)

    else:
        # Reset everything when no face is detected
        blinkCounter = 0
        # Reset timer and blink counter
        timer_started = False
        flagTimer = True 
    
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window named 'Image'
    cv2.imshow("Image", img)

    # Wait for 1 millisecond to check for any user input, keeping the window open
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()
